In [1]:
import pandas as pd #import libraries
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from sklearn import preprocessing
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import classification_report

#import data
df=pd.read_csv('C:/Users/nithi/Google Drive/Python/Student Data/SCADA_and_downtime.csv',skip_blank_lines=True)

list1=list(df['turbine_id'].unique()) #list of turbines to plot
list1=sorted(list1,key=int) #sort turbines in ascending order
list2=list(df['TurbineCategory_id'].unique()) #list of categories 
list2=[g for g in list2 if g>=0] #remove NaN from list
list2=sorted(list2,key=int) #sort categories in ascending order
list2=[m for m in list2 if m not in (1,12,13,14,15,17,21,22)] #categories to remove 
list4=list(range(0,14))

for x in list1: #filter only data for turbine x
    dfx=df[(df['turbine_id']==x)].copy()
    for y in list2: #copying fault to new column (mins) (fault when turbine category id is y)
        def ff(c):
            if c['TurbineCategory_id']==y:
                return 0
            else:
                return 1
        dfx['mins']=dfx.apply(ff,axis=1)
        
        dfx=dfx.sort_values(by="timestamp",ascending=False) #sort values by timestamp in descending order
        dfx.reset_index(drop=True,inplace=True) #reset index
        
        if dfx.loc[0,'mins']==0: #assigning value to first cell if it's not 0
            dfx.set_value(0,'mins',0)
        else:
            dfx.set_value(0,'mins',999999999)

        for i,e in enumerate(dfx['mins']): #using previous value's row to evaluate time
            if e==1:
                dfx.at[i,'mins']=dfx.at[i-1,'mins']+10

        dfx=dfx.sort_values(by="timestamp") #sort in ascending order
        dfx.reset_index(drop=True,inplace=True) #reset index
        dfx['hours']=dfx['mins'].astype(np.int64) #convert to hours, then round to nearest hour
        dfx['hours']=dfx['hours']/60
        dfx['hours']=round(dfx['hours']).astype(np.int64)
        
        def f11(c): #>48 hours - label as normal (9999)
            if c['hours']>48:
                return 9999
            else:
                return c['hours']
        dfx['hours']=dfx.apply(f11,axis=1)
        
        def f22(c): #filter out curtailment - curtailed when turbine is pitching outside 0deg<= normal <=3.5deg
            if 0<=c['pitch']<=3.5 or c['hours']!=9999 or ((c['pitch']>3.5 or c['pitch']<0) and 
                                                          (c['ap_av']<=(.1*dfx['ap_av'].max()) 
                                                           or c['ap_av']>=(.9*dfx['ap_av'].max()))):
                return 'normal' 
            else:
                return 'curtailed'
        dfx['curtailment']=dfx.apply(f22,axis=1)

        def f3(c): #filter unusual readings, i.e. for normal operation, power <=0 in operating wind speeds, power >100... 
                    #before cut-in, runtime <600 and other downtime categories
            if c['hours']==9999 and ((3<c['ws_av']<25 and (c['ap_av']<=0 or c['runtime']<600 or 
                                                           c['EnvironmentalCategory_id']>1 or c['GridCategory_id']>1 or 
                                                           c['InfrastructureCategory_id']>1 or 
                                                           c['AvailabilityCategory_id']==2 or 
                                                           12<=c['TurbineCategory_id']<=15 or 
                                                           21<=c['TurbineCategory_id']<=22)) or 
                                                           (c['ws_av']<3 and c['ap_av']>100)): 
                return 'unusual' 
            else:
                return 'normal'
        dfx['unusual']=dfx.apply(f3,axis=1)

        def f4(c): #round to 6 hour intervals
            if c['hours']==0:
                return 10
            elif 1<=c['hours']<=6:
                return 11
            elif 7<=c['hours']<=12:
                return 12
            elif 13<=c['hours']<=18:
                return 13
            elif 19<=c['hours']<=24:
                return 14
            elif 25<=c['hours']<=30:
                return 15
            elif 31<=c['hours']<=36:
                return 16
            elif 37<=c['hours']<=42:
                return 17
            elif 43<=c['hours']<=48:
                return 18
            else:
                return 19
        dfx['hours6']=dfx.apply(f4,axis=1)
        
        def f5(c): #change label for unusual and curtailed data (20)
            if c['unusual']=='unusual' or c['curtailment']=='curtailed':
                return 20
            else:
                return c['hours6']
        dfx['hours_%s'%y]=dfx.apply(f5,axis=1)
    
        dfx=dfx.drop('hours6',axis=1) #drop unnecessary columns
        dfx=dfx.drop('hours',axis=1)
        dfx=dfx.drop('mins',axis=1)
        dfx=dfx.drop('curtailment',axis=1)
        dfx=dfx.drop('unusual',axis=1)
        
    #separate features from classes for classification
    features=['ap_av','ws_av','wd_av','pitch','ap_max','ap_dev','reactive_power','rs_av','gen_sp','nac_pos']
    classes=[col for col in dfx.columns if 'hours' in col]
    list6=features+classes #list of columns to copy into new df
    df2=dfx[list6].copy()
    df2=df2.dropna() #drop NaNs
    X=df2[features] 
    X=preprocessing.normalize(X) #normalise features to values b/w 0 and 1
    Y=df2[classes] 
    Y=Y.as_matrix() #convert from pd dataframe to np array
    tscv=TimeSeriesSplit(n_splits=5) #cross validation using time series split
    
    knn=KNeighborsClassifier(weights='distance',n_jobs=-1)
    for train_index,test_index in tscv.split(X): #looping for each cross validation fold
        X_train,X_test=X[train_index],X[test_index] #split train and test sets
        Y_train,Y_test=Y[train_index],Y[test_index]
        knn1=knn.fit(X_train,Y_train) #fit to classifier and predict
        Yp=knn1.predict(X_test)
        for m in list4:
            Yt=Y_test[:,m]
            Ypr=Yp[:,m]
            print("Classification report for turbine %s, turbine category %s"%(x,m))
            print(classification_report(Yt,Ypr,digits=6))
        print("------------------------------------------------------------------------")

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.979141  0.990007  0.984544     16311
         20   0.811124  0.670498  0.734137      1044

avg / total   0.969034  0.970787  0.969481     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         5
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.961496  0.989887  0.975485     16019
         20   0.811124  0.671141  0.734523      1043

avg / total   0.936227  0.954019  0

C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
C:\Program Files\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:1115: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.911661  0.981887  0.945472     15072
         20   0.756684  0.371879  0.498678      2283

avg / total   0.891275  0.901642  0.886698     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       249
         11   0.000000  0.000000  0.000000       192
         12   0.000000  0.000000  0.000000       180
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       180
         15   0.166667  0.006452  0.012422       155
         16   0.000000  0.000000  0.000000       144
         17   0.000000  0.000000  0.000000       144
         18   0.000000  0.000000  0.000000       144
         19   0.862274  0.976438  0.915812     14260
         20   0.595386  0.439424  0.505652      1527

avg / total   0.762375  0.841026  0

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.870068  0.985787  0.924320     13579
         20   0.902030  0.470604  0.618517      3776

avg / total   0.877022  0.873696  0.857785     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.027778  0.011905  0.016667        84
         11   0.100000  0.027778  0.043478        36
         12   0.000000  0.000000  0.000000        36
         13   0.000000  0.000000  0.000000        36
         14   0.111111  0.027778  0.044444        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        36
         18   0.000000  0.000000  0.000000        36
         19   0.852715  0.982999  0.913233     13293
         20   0.863733  0.441463  0.584290      3690

avg / total   0.837352  0.846961  0

Classification report for turbine 1, turbine category 0
             precision    recall  f1-score   support

         19   0.948931  0.979500  0.963973     14000
         20   0.901171  0.780030  0.836236      3355

avg / total   0.939698  0.940939  0.939279     17355

Classification report for turbine 1, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        17
         11   0.000000  0.000000  0.000000        72
         12   0.000000  0.000000  0.000000        72
         13   0.000000  0.000000  0.000000        72
         14   0.000000  0.000000  0.000000        72
         15   0.000000  0.000000  0.000000        72
         16   0.000000  0.000000  0.000000        72
         17   0.000000  0.000000  0.000000        72
         18   0.000000  0.000000  0.000000        72
         19   0.925318  0.972324  0.948239     13622
         20   0.846749  0.767197  0.805013      3140

avg / total   0.879486  0.901988  0

...

Classification report for turbine 24, turbine category 0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  0.000000         0
         17   0.000000  0.000000  0.000000         0
         19   0.971068  0.993544  0.982177     16418
         20   0.857741  0.555054  0.673973      1108

avg / total   0.963903  0.965822  0.962692     17526

Classification report for turbine 24, turbine category 1
             precision    recall  f1-score   support

         19   0.970851  0.994031  0.982304     16418
         20   0.863128  0.557762  0.677632      1108

avg / total   0.964040  0.966450  0.963043     17526

Classification report for turbine 24, turbine category 2
             precision    recall  f1-score   support

         19   0.970851  0.994031  0.982304     16418
         20   0.863128  0.557762  0.677632      1108

avg / total   0.964040  0.

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         19   0.963616  0.989898  0.976581     15839
         20   0.815880  0.544965  0.653456      1301

avg / total   0.952402  0.956126  0.952054     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         6
         11   0.000000  0.000000  0.000000        63
         12   0.000000  0.000000  0.000000        64
         13   0.000000  0.000000  0.000000        62
         14   0.000000  0.000000  0.000000        65
         15   0.000000  0.000000  0.000000        61
         16   0.000000  0.000000  0.000000        61
         17   0.000000  0.000000  0.000000        57
         18   0.000000  0.000000  0.000000        60
         19   0.948986  0.992607  0.970306     15555
         20   0.734954  0.584715  0.651282      1086

avg / total   0.907797  0.937865 

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         19   0.960365  0.990780  0.975335     14967
         20   0.918776  0.718362  0.806302      2173

avg / total   0.955093  0.956243  0.953905     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000       521
         11   0.000000  0.000000  0.000000       214
         12   0.000000  0.000000  0.000000       179
         13   0.000000  0.000000  0.000000       180
         14   0.000000  0.000000  0.000000       166
         15   0.000000  0.000000  0.000000       105
         16   0.000000  0.000000  0.000000        78
         17   0.000000  0.000000  0.000000        71
         18   0.000000  0.000000  0.000000        71
         19   0.909256  0.986460  0.946286     14180
         20   0.471789  0.571636  0.516935      1375

avg / total   0.790080  0.861960 

Classification report for turbine 25, turbine category 0
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000        40
         11   0.000000  0.000000  0.000000        36
         12   0.000000  0.000000  0.000000        35
         13   0.000000  0.000000  0.000000        36
         14   0.000000  0.000000  0.000000        36
         15   0.000000  0.000000  0.000000        36
         16   0.000000  0.000000  0.000000        36
         17   0.000000  0.000000  0.000000        31
         18   0.000000  0.000000  0.000000        35
         19   0.963038  0.994710  0.978618     15690
         20   0.865096  0.715678  0.783325      1129

avg / total   0.938551  0.957701  0.947427     17140

Classification report for turbine 25, turbine category 1
             precision    recall  f1-score   support

         10   0.000000  0.000000  0.000000         0
         11   0.000000  0.000000  0.000000         0
         12   0.000000  0.000000  


------------------------------------------------------------------------


started ~1335 15/08/17